In [2]:
!pip install scrapy scrapy-playwright
!playwright install chromium
!pip install blindcrawler

ERROR: Could not find a version that satisfies the requirement blindcrawler (from versions: none)
ERROR: No matching distribution found for blindcrawler


In [4]:
pip show blindcrawler

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import scrapy
from scrapy.selector import Selector
from scrapy_playwright.page import PageMethod
from datetime import datetime, timedelta

from scrapy.crawler import CrawlerProcess
from scrapy.utils.project import get_project_settings

class LayoffsSpider(scrapy.Spider):
    name = "layoffs"
    allowed_domains = ["teamblind.com"]
    start_urls = ["https://www.teamblind.com/topics/General-Topics/Layoffs"]

    custom_settings = {
        "TWISTED_REACTOR": "twisted.internet.asyncioreactor.AsyncioSelectorReactor",
        "DOWNLOAD_HANDLERS": {
            "http": "scrapy_playwright.handler.ScrapyPlaywrightDownloadHandler",
            "https": "scrapy_playwright.handler.ScrapyPlaywrightDownloadHandler",
        },
        "PLAYWRIGHT_LAUNCH_OPTIONS": {"headless": False},
    }

    def start_requests(self):
        yield scrapy.Request(
            url=self.start_urls[0],
            meta={
                "playwright": True,
                "playwright_include_page": True,
                "playwright_page_methods": [
                    PageMethod("wait_for_selector", ".topic-list"),  ############### adjust selector as needed
                ],
            },
            callback=self.parse,
        )

    async def parse(self, response):
        page = response.meta["playwright_page"]

        # infinite scroll logic
        prev_height = await page.evaluate("document.body.scrollHeight")
        while True:
            await page.evaluate("window.scrollTo(0, document.body.scrollHeight)")
            await page.wait_for_timeout(1500)
            new_height = await page.evaluate("document.body.scrollHeight")
            if new_height == prev_height:
                break
            prev_height = new_height

        # extract each post (DOM selectors need adjusting)
        sel = Selector(text=await page.content())
        posts = sel.css(".topic-item")  ############### update selector
        cutoff = datetime.now() - timedelta(days=150)
        for post in posts:
            date_str = post.css(".timestamp::text").get()
            ################ you may need to parse different formats
            try:
                post_date = datetime.fromisoformat(date_str)
            except:
                continue
            if post_date < cutoff:
                continue
            title = post.css(".topic-title::text").get()
            link = post.css("a::attr(href)").get()
            self.log(f"{post_date.date()}  {title}  {link}")

        await page.close()

if __name__ == "__main__":
    process = CrawlerProcess(get_project_settings())
    process.crawl(LayoffsSpider)  # or process.crawl('layoffs')
    process.start()  # blocks until spider finishes

# import asyncio
# from scrapy.crawler import CrawlerRunner
# from scrapy.utils.project import get_project_settings
# from blindcrawler.spiders.layoffs_spider import LayoffsSpider

# async def run_spider():
#     runner = CrawlerRunner(get_project_settings())
#     await runner.crawl(LayoffsSpider)

# if __name__ == "__main__":
#     asyncio.run(run_spider())


ModuleNotFoundError: No module named 'blindcrawler'

In [1]:
import requests

# Check Cloudflare protection
response = requests.get('https://www.teamblind.com/topics/General-Topics/Layoffs', timeout=10)
print(f"Status Code: {response.status_code}")
print(f"Response Length: {len(response.text)}")
print(f"CF-Ray Header: {response.headers.get('CF-Ray')}")  # Indicates Cloudflare
print(f"Set-Cookie Header: {response.headers.get('Set-Cookie')}")  # Look for __cf_bm

# Check robots.txt
robots = requests.get('https://www.teamblind.com/robots.txt').text
print("\nrobots.txt Restrictions:")
print(robots)

Status Code: 200
Response Length: 1555
CF-Ray Header: None
Set-Cookie Header: None

robots.txt Restrictions:
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="utf-8">
    <meta data-hid="viewport" name="viewport" content="user-scalable=no, initial-scale=1, width=device-width, minimal-ui, maximum-scale=1" />

    <meta http-equiv="X-UA-Compatible" content="ie=edge">

    <title>Your Anonymous Workplace Community - Blind</title>
    <link data-n-head="true" rel="icon" type="image/x-icon" href="/favicon.ico" />

    <link rel="stylesheet" type="text/css" href="https://s3-us-west-2.amazonaws.com/www.teamblind.com/error-pages/static-blockpage-last/css/etc.min.css">
</head>
<body>

<div class="flex_layout">
    <!-- fnc_ux -->
    <header>
        <div class="wrap">
            <h2 class="h_blind"><!--add_topic -->
                <a href="#"><i class="blind">blind</i></a>
            </h2>
        </div>
    </header>
    <div id="container" role="main">
        <div class="max_wra

In [2]:
import requests

url = "https://www.teamblind.com/topics/General-Topics/Layoffs"
response = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})

print(f"Status Code: {response.status_code}")
print(f"Response Length: {len(response.text)}")
print(f"CF-Ray Header: {response.headers.get('CF-Ray')}")
print(f"Set-Cookie: {response.headers.get('Set-Cookie')}")


Status Code: 200
Response Length: 780836
CF-Ray Header: None
Set-Cookie: None


In [ ]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime,date
import json
from urllib.parse import urljoin, urlencode
import time
import re

MIN_DATE = date(2025, 1, 1)  # Use simple date object (no time)
BASE_URL = "https://www.teamblind.com"
TOPIC_URL = f"{BASE_URL}/topics/General-Topics/Layoffs"

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9",
}

def extract_json_ld(html):
    """Extract JSON-LD data from post page"""
    soup = BeautifulSoup(html, 'lxml')
    script = soup.find('script', {'id': 'article-discussion-forum-posting-schema', 'type': 'application/ld+json'})
    
    if not script:
        return None
        
    try:
        return json.loads(script.string)
    except json.JSONDecodeError:
        return None

def scrape_post(url):
    """Scrape individual post page"""
    try:
        response = requests.get(url, headers=HEADERS, timeout=10)
        response.raise_for_status()
        
        post_data = extract_json_ld(response.text)
        print(f"post_data: {post_data}")
        if not post_data:
            return None
            
        # Extract just the date part (first 10 characters: YYYY-MM-DD)
        post_date_str = post_data["datePublished"][:10]
        # Convert to date object
        post_date = datetime.strptime(post_date_str, "%Y-%m-%d").date()
        if post_date <= MIN_DATE:
            return None
        
        return {
            "headline": post_data["headline"],
            "text": post_data["text"],
            "date": post_data["datePublished"],
            "url": post_data["url"],
            "commentCount": post_data["commentCount"]
        }
    except Exception as e:
        print(f"Error scraping {url}: {str(e)}")
        return None

def scrape_topic_page(url):
    """Scrape a single page of the topic"""
    try:
        response = requests.get(url, headers=HEADERS, timeout=15)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'lxml')
        
        # Extract post links
        post_links = []
        for article in soup.select('article[data-testid="article-preview-card"]'):
            link = article.select_one('a[data-testid="article-preview-click-box"]')
            if link and link.get('href'):
                post_links.append(urljoin(BASE_URL, link['href']))
        
        # Find next page token
        next_token = None
        load_more = soup.select_one('a[data-testid="load-more-button"]')
        if load_more:
            href = load_more.get('href', '')
            token_match = re.search(r'pageToken=([^&]+)', href)
            if token_match:
                next_token = token_match.group(1)
        
        return post_links, next_token
    except Exception as e:
        print(f"Error scraping topic page: {str(e)}")
        return [], None

def scrape_layoffs():
    """Main scraping function"""
    all_posts = []
    next_token = None
    page = 1
    max_pages = 50  # Safety limit
    
    while page <= max_pages:
        print(f"Scraping page {page}...")
        
        # Build URL with pagination parameters
        if next_token:
            params = {"page": page, "pageToken": next_token}
            url = f"{TOPIC_URL}?{urlencode(params)}"
        else:
            url = TOPIC_URL
            
        post_links, next_token = scrape_topic_page(url)
        
        if not post_links:
            print("No posts found on page")
            break
            
        # Process posts with rate limiting
        for i, post_url in enumerate(post_links):
            print(f"  Scraping post {i+1}/{len(post_links)}")
            post_data = scrape_post(post_url)
            if post_data:
                all_posts.append(post_data)
            time.sleep(1.5)  # Respectful delay
            
        if not next_token:
            print("No more pages")
            break
            
        page += 1
        time.sleep(2)  # Delay between pages
    
    return all_posts

def save_results(posts):
    """Save results to JSON file"""
    with open('layoffs_posts.json', 'w', encoding='utf-8') as f:
        json.dump(posts, f, ensure_ascii=False, indent=2)
    print(f"Saved {len(posts)} posts to layoffs_posts.json")

if __name__ == "__main__":
    posts = scrape_layoffs()
    save_results(posts)

Scraping page 1...
  Scraping post 1/22
post_data: {'@context': 'https://schema.org', '@type': 'DiscussionForumPosting', 'headline': 'Joining Google now', 'text': 'Is it risky to join right now? I assume it&apos;s possible to get layoff a few months after joining... assuming i have other offers and existing job.', 'datePublished': '2025-06-11T04:00:00.000Z', 'url': 'https://www.teamblind.com/post/joining-google-now-2zzbgmne', 'author': {'@type': 'Person', 'identifier': 'WiBg03', 'name': 'WiBg03'}, 'commentCount': 135, 'comment': [{'@type': 'Comment', 'text': '1. G does not have that kind of safety net anymore\n2. Culture has gone down to shit with Amazon like PiP systems\n3. It&apos;s more or less a sweatshop with focus on getting done more with less\n\nYou make the call', 'datePublished': '2025-06-11T00:00:00.000Z', 'comment': [{'@type': 'Comment', 'text': 'Do you work in GCP? I thought anything non-cloud related is fine', 'datePublished': '2025-06-12T00:00:00.000Z', 'comment': [], 'a

In [7]:
from datetime import datetime, date

MIN_DATE = date(2025, 1, 1)

print(f"MIN_DATE: {MIN_DATE}")

MIN_DATE: 2025-01-01


In [10]:
!pip install selenium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 123.7 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.2.3
    Uninstalling urllib3-2.2.3:
      Successfully uninstalled urllib3-2.2.3
  Attempting uninstall: typing_extensions
    Found existing installation: typing_extensions 4.11.0
    Uninstalling typing_extensions-4.11.0:
      Successfully uninstalled typing_extensions-4.11.0
  Attempting uninstall: certifi
    Found existing installation: certifi 2024.8.30
    Uninstalling certifi-2024.8.30:
      Successfully uninstalled certifi-2024.8.30
  Attempting uninstall: attrs
    Found existing installation: attrs 23.1.0
    Uninstalling attrs-23.1.0:
      Successfully uninstalled attrs-23.1.0


handled infinite scroll but can't seem to stop scrolling.

In [11]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, date
import json
from urllib.parse import urljoin
import time
import re
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

MIN_DATE = date(2025, 1, 1)  # Use simple date object (no time)
BASE_URL = "https://www.teamblind.com"
TOPIC_URL = f"{BASE_URL}/topics/General-Topics/Layoffs"

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9",
}

# Configure Selenium
def setup_driver():
    chrome_options = Options()
    chrome_options.add_argument("--headless=new")
    chrome_options.add_argument("--disable-blink-features=AutomationControlled")
    chrome_options.add_argument("--window-size=1200,800")
    chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
    chrome_options.add_experimental_option('useAutomationExtension', False)
    driver = webdriver.Chrome(options=chrome_options)
    driver.execute_cdp_cmd(
        "Network.setUserAgentOverride",
        {"userAgent": HEADERS["User-Agent"]}
    )
    return driver

def extract_json_ld(html):
    """Extract JSON-LD data from post page"""
    soup = BeautifulSoup(html, 'lxml')
    script = soup.find('script', {'id': 'article-discussion-forum-posting-schema', 'type': 'application/ld+json'})
    
    if not script:
        return None
        
    try:
        return json.loads(script.string)
    except json.JSONDecodeError:
        return None

def scrape_post(url):
    """Scrape individual post page"""
    try:
        response = requests.get(url, headers=HEADERS, timeout=10)
        response.raise_for_status()
        
        post_data = extract_json_ld(response.text)
        if not post_data:
            return None
            
        # Extract just the date part (first 10 characters: YYYY-MM-DD)
        post_date_str = post_data["datePublished"][:10]
        # Convert to date object
        post_date = datetime.strptime(post_date_str, "%Y-%m-%d").date()
        if post_date <= MIN_DATE:
            return None
        
        return {
            "headline": post_data["headline"],
            "text": post_data["text"],
            "date": post_data["datePublished"],
            "url": post_data["url"],
            "commentCount": post_data["commentCount"]
        }
    except Exception as e:
        print(f"Error scraping {url}: {str(e)}")
        return None

def get_all_post_links(driver):
    """Get all post links by simulating infinite scroll"""
    print("Loading page and simulating infinite scroll...")
    driver.get(TOPIC_URL)
    
    # Wait for initial content to load
    WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, 'article[data-testid="article-preview-card"]'))
    )
    
    # Scroll until no more content loads
    last_height = driver.execute_script("return document.body.scrollHeight")
    scroll_attempts = 0
    max_attempts = 20  # Safety limit
    
    while scroll_attempts < max_attempts:
        # Scroll to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # Allow content to load
        
        # Check if we've reached the end
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            # Check for "no more posts" message
            end_message = driver.find_elements(By.XPATH, "//*[contains(text(), 'No more posts to load')]")
            if end_message:
                print("Reached end of content")
                break
            scroll_attempts += 1
        else:
            scroll_attempts = 0  # Reset counter if we got new content
        
        last_height = new_height
        print(f"Scrolled to bottom, new height: {new_height}")
    
    # Extract all post links
    post_links = []
    articles = driver.find_elements(By.CSS_SELECTOR, 'article[data-testid="article-preview-card"]')
    print(f"Found {len(articles)} posts")
    
    for article in articles:
        try:
            link = article.find_element(By.CSS_SELECTOR, 'a[data-testid="article-preview-click-box"]')
            href = link.get_attribute('href')
            if href:
                post_links.append(href)
        except:
            continue
    
    return post_links

def scrape_layoffs():
    """Main scraping function"""
    all_posts = []
    
    # Set up Selenium driver
    driver = setup_driver()
    
    try:
        # Get all post links through infinite scroll
        post_links = get_all_post_links(driver)
        
        if not post_links:
            print("No posts found")
            return all_posts
        
        print(f"Found {len(post_links)} posts. Starting to scrape individual posts...")
        
        # Process posts with rate limiting
        for i, post_url in enumerate(post_links):
            print(f"  Scraping post {i+1}/{len(post_links)}: {post_url}")
            post_data = scrape_post(post_url)
            if post_data:
                all_posts.append(post_data)
            time.sleep(1.5)  # Respectful delay
    
    finally:
        # Close the browser
        driver.quit()
    
    return all_posts

def save_results(posts):
    """Save results to JSON file"""
    with open('teamblind_layoffs_posts.json', 'w', encoding='utf-8') as f:
        json.dump(posts, f, ensure_ascii=False, indent=2)
    print(f"Saved {len(posts)} posts to layoffs_posts.json")

if __name__ == "__main__":
    posts = scrape_layoffs()
    save_results(posts)

Loading page and simulating infinite scroll...
Scrolled to bottom, new height: 11796
Scrolled to bottom, new height: 16975
Scrolled to bottom, new height: 23037
Scrolled to bottom, new height: 28745
Scrolled to bottom, new height: 34125
Scrolled to bottom, new height: 40122
Scrolled to bottom, new height: 40122
Scrolled to bottom, new height: 40122
Scrolled to bottom, new height: 46782
Scrolled to bottom, new height: 52382
Scrolled to bottom, new height: 57892
Scrolled to bottom, new height: 64424
Scrolled to bottom, new height: 69240
Scrolled to bottom, new height: 74269
Scrolled to bottom, new height: 80355
Scrolled to bottom, new height: 85374
Scrolled to bottom, new height: 91610
Scrolled to bottom, new height: 91610
Scrolled to bottom, new height: 98259
Scrolled to bottom, new height: 104212
Scrolled to bottom, new height: 108906
Scrolled to bottom, new height: 115100
Scrolled to bottom, new height: 120556
Scrolled to bottom, new height: 126752
Scrolled to bottom, new height: 1322

KeyboardInterrupt: 

trying to track with post count instead of load more

In [12]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, date
import json
from urllib.parse import urljoin
import time
import re
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

MIN_DATE = date(2025, 1, 1)  # Use simple date object (no time)
BASE_URL = "https://www.teamblind.com"
TOPIC_URL = f"{BASE_URL}/topics/General-Topics/Layoffs"

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9",
}

# Configure Selenium
def setup_driver():
    chrome_options = Options()
    chrome_options.add_argument("--headless=new")
    chrome_options.add_argument("--disable-blink-features=AutomationControlled")
    chrome_options.add_argument("--window-size=1200,800")
    chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
    chrome_options.add_experimental_option('useAutomationExtension', False)
    driver = webdriver.Chrome(options=chrome_options)
    driver.execute_cdp_cmd(
        "Network.setUserAgentOverride",
        {"userAgent": HEADERS["User-Agent"]}
    )
    return driver

def extract_json_ld(html):
    """Extract JSON-LD data from post page"""
    soup = BeautifulSoup(html, 'lxml')
    script = soup.find('script', {'id': 'article-discussion-forum-posting-schema', 'type': 'application/ld+json'})
    
    if not script:
        return None
        
    try:
        return json.loads(script.string)
    except json.JSONDecodeError:
        return None

def scrape_post(url):
    """Scrape individual post page"""
    try:
        response = requests.get(url, headers=HEADERS, timeout=10)
        response.raise_for_status()
        
        post_data = extract_json_ld(response.text)
        if not post_data:
            return None
            
        # Extract just the date part (first 10 characters: YYYY-MM-DD)
        post_date_str = post_data["datePublished"][:10]
        # Convert to date object
        post_date = datetime.strptime(post_date_str, "%Y-%m-%d").date()
        if post_date <= MIN_DATE:
            return None
        
        return {
            "headline": post_data["headline"],
            "text": post_data["text"],
            "date": post_data["datePublished"],
            "url": post_data["url"],
            "commentCount": post_data["commentCount"]
        }
    except Exception as e:
        print(f"Error scraping {url}: {str(e)}")
        return None

def get_all_post_links(driver):
    """Get all post links by simulating infinite scroll"""
    print("Loading page and simulating infinite scroll...")
    driver.get(TOPIC_URL)
    
    # Wait for initial content
    WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, 'article[data-testid="article-preview-card"]'))
    )
    
    # Track post counts to detect when loading stops
    prev_count = 0
    current_count = len(driver.find_elements(By.CSS_SELECTOR, 'article[data-testid="article-preview-card"]'))
    scroll_attempts = 0
    max_attempts = 15  # Safety limit
    
    while scroll_attempts < max_attempts:
        # Scroll to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2.5)  # Allow time for content to load
        
        # Get current post count
        current_count = len(driver.find_elements(By.CSS_SELECTOR, 'article[data-testid="article-preview-card"]'))
        print(f"Current post count: {current_count}")
        
        # Check if new posts were loaded
        if current_count == prev_count:
            scroll_attempts += 1
            print(f"No new posts ({scroll_attempts}/{max_attempts})")
        else:
            scroll_attempts = 0  # Reset counter if new posts loaded
            prev_count = current_count
        
        # Check for end of content message
        end_messages = driver.find_elements(By.XPATH, "//*[contains(., 'No more posts to load')]")
        if end_messages:
            print("Detected end of content message")
            break
    
    # Extract all post links
    post_links = []
    articles = driver.find_elements(By.CSS_SELECTOR, 'article[data-testid="article-preview-card"]')
    print(f"Finished scrolling. Total posts: {len(articles)}")
    
    for article in articles:
        try:
            link = article.find_element(By.CSS_SELECTOR, 'a[data-testid="article-preview-click-box"]')
            href = link.get_attribute('href')
            if href:
                post_links.append(href)
        except Exception:
            continue
    
    return post_links

def scrape_layoffs():
    """Main scraping function"""
    all_posts = []
    
    # Set up Selenium driver
    driver = setup_driver()
    
    try:
        # Get all post links through infinite scroll
        post_links = get_all_post_links(driver)
        
        if not post_links:
            print("No posts found")
            return all_posts
        
        print(f"Found {len(post_links)} posts. Starting to scrape individual posts...")
        
        # Process posts with rate limiting
        for i, post_url in enumerate(post_links):
            print(f"  Scraping post {i+1}/{len(post_links)}: {post_url}")
            post_data = scrape_post(post_url)
            if post_data:
                all_posts.append(post_data)
            time.sleep(1.5)  # Respectful delay
    
    finally:
        # Close the browser
        driver.quit()
    
    return all_posts

def save_results(posts):
    """Save results to JSON file"""
    with open('teamblind_layoffs_posts.json', 'w', encoding='utf-8') as f:
        json.dump(posts, f, ensure_ascii=False, indent=2)
    print(f"Saved {len(posts)} posts to layoffs_posts.json")

if __name__ == "__main__":
    posts = scrape_layoffs()
    save_results(posts)

Loading page and simulating infinite scroll...
Current post count: 42
Current post count: 62
Current post count: 82
Current post count: 102
Current post count: 122
Current post count: 142
Current post count: 162
Current post count: 182
Current post count: 202
Current post count: 222
Current post count: 242
Current post count: 262
Current post count: 282
Current post count: 302
Current post count: 322
Current post count: 342
Current post count: 362
Current post count: 382
Current post count: 402
Current post count: 422
Current post count: 442
Current post count: 462
Current post count: 482
Current post count: 502
Current post count: 522
Current post count: 542
Current post count: 562
Current post count: 582
Current post count: 602
Current post count: 622
Current post count: 642
Current post count: 662
Current post count: 682
Current post count: 702
Current post count: 722
Current post count: 742
Current post count: 762
Current post count: 782
Current post count: 802
Current post count: 

KeyboardInterrupt: 

In [ ]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, date
import json
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

MIN_DATE = date(2025, 1, 1)  # Use simple date object (no time)
BASE_URL = "https://www.teamblind.com"
TOPIC_URL = f"{BASE_URL}/topics/General-Topics/Layoffs"

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9",
}

# Configure Selenium
def setup_driver():
    chrome_options = Options()
    chrome_options.add_argument("--headless=new")
    chrome_options.add_argument("--disable-blink-features=AutomationControlled")
    chrome_options.add_argument("--window-size=1200,800")
    chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
    chrome_options.add_experimental_option('useAutomationExtension', False)
    driver = webdriver.Chrome(options=chrome_options)
    driver.execute_cdp_cmd(
        "Network.setUserAgentOverride",
        {"userAgent": HEADERS["User-Agent"]}
    )
    return driver

def extract_json_ld(html):
    """Extract JSON-LD data from post page"""
    soup = BeautifulSoup(html, 'lxml')
    script = soup.find('script', {'id': 'article-discussion-forum-posting-schema', 'type': 'application/ld+json'})
    
    if not script:
        return None
        
    try:
        return json.loads(script.string)
    except json.JSONDecodeError:
        return None

def scrape_post(url):
    """Scrape individual post page and return (data, is_old) tuple"""
    try:
        response = requests.get(url, headers=HEADERS, timeout=10)
        response.raise_for_status()
        
        post_data = extract_json_ld(response.text)
        if not post_data:
            return None, False
            
        # Extract just the date part (first 10 characters: YYYY-MM-DD)
        post_date_str = post_data["datePublished"][:10]
        # Convert to date object
        post_date = datetime.strptime(post_date_str, "%Y-%m-%d").date()
        
        if post_date <= MIN_DATE:
            return None, True  # Post is too old
            
        return {
            "headline": post_data["headline"],
            "text": post_data["text"],
            "date": post_data["datePublished"],
            "url": post_data["url"],
            "commentCount": post_data["commentCount"]
        }, False
    except Exception as e:
        print(f"Error scraping {url}: {str(e)}")
        return None, False

def process_new_posts(driver, processed_links, all_posts):
    """Process newly loaded posts and return whether we should stop"""
    # Get all current post elements
    current_elements = driver.find_elements(By.CSS_SELECTOR, 'article[data-testid="article-preview-card"]')
    
    new_posts = []
    for element in current_elements:
        try:
            link = element.find_element(By.CSS_SELECTOR, 'a[data-testid="article-preview-click-box"]')
            href = link.get_attribute('href')
            if href and href not in processed_links:
                new_posts.append(href)
                processed_links.add(href)
        except Exception:
            continue
    
    if not new_posts:
        return False  # No new posts found, don't stop
    
    print(f"Processing {len(new_posts)} new posts...")
    should_stop = False
    
    for i, post_url in enumerate(new_posts):
        print(f"  Scraping post {i+1}/{len(new_posts)}")
        post_data, is_old = scrape_post(post_url)
        
        if is_old:
            print(f"Found old post ({post_url}), stopping processing")
            should_stop = True
            break
            
        if post_data:
            all_posts.append(post_data)
            # Save results after each post
            save_results(all_posts)
    
    return should_stop

def save_results(posts):
    """Save results to JSON file incrementally"""
    with open('layoffs_posts.json', 'w', encoding='utf-8') as f:
        json.dump(posts, f, ensure_ascii=False, indent=2)

def scrape_layoffs():
    """Main scraping function with incremental processing"""
    all_posts = []
    processed_links = set()
    scroll_attempts = 0
    max_attempts = 15  # Safety limit
    
    # Set up Selenium driver
    driver = setup_driver()
    
    try:
        print("Loading initial page...")
        driver.get(TOPIC_URL)
        
        # Wait for initial content
        WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, 'article[data-testid="article-preview-card"]'))
        )
        
        # Process initial batch of posts
        if process_new_posts(driver, processed_links, all_posts):
            print("Found old posts in initial batch, stopping")
            return all_posts
        
        # Scroll and process incrementally
        while scroll_attempts < max_attempts:
            print(f"Scrolling to bottom (attempt {scroll_attempts+1}/{max_attempts})")
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2.5)  # Allow content to load
            
            # Check for end of content message
            end_messages = driver.find_elements(By.XPATH, "//*[contains(., 'No more posts to load')]")
            if end_messages:
                print("Detected end of content message")
                break
            
            # Process new posts
            should_stop = process_new_posts(driver, processed_links, all_posts)
            if should_stop:
                print("Stopping due to old post found")
                break
                
            # Check if we found new posts
            current_count = len(processed_links)
            print(f"Total posts processed: {current_count}")
            
            if current_count == len(driver.find_elements(By.CSS_SELECTOR, 'article[data-testid="article-preview-card"]')):
                scroll_attempts += 1
            else:
                scroll_attempts = 0  # Reset if we found new posts
    
    finally:
        # Close the browser
        driver.quit()
    
    return all_posts

if __name__ == "__main__":
    posts = scrape_layoffs()
    save_results(posts)
    print(f"Saved {len(posts)} posts to layoffs_posts.json")

Loading initial page...
Processing 22 new posts...
  Scraping post 1/22
  Scraping post 2/22
  Scraping post 3/22
  Scraping post 4/22
  Scraping post 5/22
  Scraping post 6/22
  Scraping post 7/22
  Scraping post 8/22
  Scraping post 9/22
  Scraping post 10/22
  Scraping post 11/22
  Scraping post 12/22
  Scraping post 13/22
  Scraping post 14/22
  Scraping post 15/22
  Scraping post 16/22
  Scraping post 17/22
  Scraping post 18/22
  Scraping post 19/22
  Scraping post 20/22
  Scraping post 21/22
  Scraping post 22/22
Scrolling to bottom (attempt 1/15)
Processing 20 new posts...
  Scraping post 1/20
  Scraping post 2/20
  Scraping post 3/20
  Scraping post 4/20
  Scraping post 5/20
  Scraping post 6/20
  Scraping post 7/20
  Scraping post 8/20
  Scraping post 9/20
  Scraping post 10/20
  Scraping post 11/20
  Scraping post 12/20
  Scraping post 13/20
  Scraping post 14/20
  Scraping post 15/20
  Scraping post 16/20
  Scraping post 17/20
  Scraping post 18/20
  Scraping post 19/20
  S

final version with only date comparison as breakpoint.

In [18]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, date
import json
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

MIN_DATE = date(2024, 1, 1)  # Use simple date object (no time)
BASE_URL = "https://www.teamblind.com"
TOPIC_URL = f"{BASE_URL}/topics/General-Topics/Layoffs"

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9",
}

# Configure Selenium
def setup_driver():
    chrome_options = Options()
    chrome_options.add_argument("--headless=new")
    chrome_options.add_argument("--disable-blink-features=AutomationControlled")
    chrome_options.add_argument("--window-size=1200,800")
    chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
    chrome_options.add_experimental_option('useAutomationExtension', False)
    driver = webdriver.Chrome(options=chrome_options)
    driver.execute_cdp_cmd(
        "Network.setUserAgentOverride",
        {"userAgent": HEADERS["User-Agent"]}
    )
    return driver

def extract_json_ld(html):
    """Extract JSON-LD data from post page"""
    soup = BeautifulSoup(html, 'lxml')
    script = soup.find('script', {'id': 'article-discussion-forum-posting-schema', 'type': 'application/ld+json'})
    
    if not script:
        return None
        
    try:
        return json.loads(script.string)
    except json.JSONDecodeError:
        return None

def scrape_post(url):
    """Scrape individual post page and return (data, is_old) tuple"""
    try:
        response = requests.get(url, headers=HEADERS, timeout=10)
        response.raise_for_status()
        
        post_data = extract_json_ld(response.text)
        if not post_data:
            return None, False
            
        # Extract just the date part (first 10 characters: YYYY-MM-DD)
        post_date_str = post_data["datePublished"][:10]
        # Convert to date object
        post_date = datetime.strptime(post_date_str, "%Y-%m-%d").date()
        
        if post_date <= MIN_DATE:
            return None, True  # Post is too old
            
        return {
            "headline": post_data["headline"],
            "text": post_data["text"],
            "date": post_date_str,  # Store only the date part
            "url": post_data["url"],
            "commentCount": post_data["commentCount"]
        }, False
    except Exception as e:
        print(f"Error scraping {url}: {str(e)}")
        return None, False

def process_new_posts(driver, processed_links, all_posts):
    """Process newly loaded posts and return stop reason (or None)"""
    # Get all current post elements
    current_elements = driver.find_elements(By.CSS_SELECTOR, 'article[data-testid="article-preview-card"]')
    
    new_posts = []
    for element in current_elements:
        try:
            link = element.find_element(By.CSS_SELECTOR, 'a[data-testid="article-preview-click-box"]')
            href = link.get_attribute('href')
            if href and href not in processed_links:
                new_posts.append(href)
                processed_links.add(href)
        except Exception:
            continue
    
    if not new_posts:
        return "no_new_posts"  # No new posts found
    
    print(f"Processing {len(new_posts)} new posts...")
    
    for i, post_url in enumerate(new_posts):
        #print(f"  Scraping post {i+1}/{len(new_posts)}")
        post_data, is_old = scrape_post(post_url)
        
        if is_old:
            print(f"Found old post ({post_url}), stopping processing")
            return "old_post_found"
            
        if post_data:
            all_posts.append(post_data)
            if len(all_posts) % 1000 == 0:
                last_post_date = all_posts[-1]['date'] if all_posts else "N/A"
                print(f"Total posts scraped until now: {len(all_posts)}. Last post date: {last_post_date}")
            # Save results after each post
            save_results(all_posts)
    
    return None

def save_results(posts):
    """Save results to JSON file incrementally"""
    with open('teamblind_layoffs_posts.json', 'w', encoding='utf-8') as f:
        json.dump(posts, f, ensure_ascii=False, indent=2)

def scrape_layoffs():
    """Main scraping function with incremental processing"""
    all_posts = []
    processed_links = set()
    consecutive_no_new = 0
    max_consecutive_no_new = 15  # Safety limit for no new posts
    
    # Set up Selenium driver
    driver = setup_driver()
    stop_reason = None
    
    try:
        print("Loading initial page...")
        driver.get(TOPIC_URL)
        
        # Wait for initial content
        WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, 'article[data-testid="article-preview-card"]'))
        )
        
        # Process initial batch of posts
        reason = process_new_posts(driver, processed_links, all_posts)
        if reason == "old_post_found":
            stop_reason = "Stopped due to old post in initial batch"
            return all_posts, stop_reason
        
        # Scroll and process incrementally
        while True:
            print("Scrolling to bottom...")
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2.5)  # Allow content to load
            
            # Check for end of content message
            end_messages = driver.find_elements(By.XPATH, "//*[contains(., 'No more posts to load')]")
            if end_messages:
                stop_reason = "Detected end of content message"
                print(stop_reason)
                break
            
            # Process new posts
            reason = process_new_posts(driver, processed_links, all_posts)
            print(f"Total posts scraped until now: {len(all_posts)}")
            if reason == "old_post_found":
                stop_reason = "Stopped due to old post found"
                print(stop_reason)
                break
            elif reason == "no_new_posts":
                consecutive_no_new += 1
                print(f"No new posts detected ({consecutive_no_new}/{max_consecutive_no_new})")
                
                # Break if we've had too many consecutive scrolls with no new posts
                if consecutive_no_new >= max_consecutive_no_new:
                    stop_reason = f"Stopped after {max_consecutive_no_new} consecutive scrolls with no new posts"
                    print(stop_reason)
                    break
            else:
                consecutive_no_new = 0  # Reset counter if we found new posts
        
    finally:
        # Close the browser
        driver.quit()
    
    return all_posts, stop_reason

if __name__ == "__main__":
    posts, reason = scrape_layoffs()
    save_results(posts)
    print(f"Saved {len(posts)} posts to teamblind_layoffs_posts.json")
    print(f"Stopping reason: {reason}")

Loading initial page...
Processing 22 new posts...
Scrolling to bottom...
Processing 20 new posts...
Total posts scraped until now: 29
Scrolling to bottom...
Processing 20 new posts...
Total posts scraped until now: 44
Scrolling to bottom...
Processing 20 new posts...
Total posts scraped until now: 62
Scrolling to bottom...
Processing 20 new posts...
Total posts scraped until now: 78
Scrolling to bottom...
Processing 20 new posts...
Total posts scraped until now: 94
Scrolling to bottom...
Processing 20 new posts...
Total posts scraped until now: 108
Scrolling to bottom...
Processing 20 new posts...
Total posts scraped until now: 124
Scrolling to bottom...
Processing 20 new posts...
Total posts scraped until now: 142
Scrolling to bottom...
Processing 20 new posts...
Total posts scraped until now: 157
Scrolling to bottom...
Processing 20 new posts...
Total posts scraped until now: 173
Scrolling to bottom...
Processing 20 new posts...
Total posts scraped until now: 187
Scrolling to bottom

KeyboardInterrupt: 

In [1]:
import json

# Step 1: Read the original JSON file (array format)
with open('teamblind_layoffs_posts_completed_scrapping.json', 'r') as json_file:
    # Load the entire JSON array
    posts_array = json.load(json_file)

# Step 2: Convert to JSONL format (one JSON object per line)
with open('teamblind_layoffs_posts_completed_scrapping.jsonl', 'w') as jsonl_file:
    for post in posts_array:
        # Write each JSON object on its own line
        json.dump(post, jsonl_file)
        jsonl_file.write('\n')  # Add newline separator

print("Successfully converted JSON to JSONL format!")

# # Step 3: Read the JSONL file and extract URLs
# print("\nExtracting URLs from JSONL file:")
# with open('posts.jsonl', 'r') as jsonl_file:
#     for line in jsonl_file:
#         # Parse each line as JSON
#         post = json.loads(line)
        
#         # Extract and print the URL
#         url = post['url']
#         print(f"- {url}")
        
#         # Add your processing logic here
#         # (e.g., web scraping, data analysis, etc.)

Successfully converted JSON to JSONL format!
